In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore')

## 신경망 모델 구성 

In [2]:
import os
import torch
# torch.nn 네임스페이스는 신경망을 구성하는데 필요한 모든 구성 요소를 제공
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


## 클래스 정의
신경망 모델을 nn.Module 의 하위클래스로 정의하고, __init__ 에서 신경망 계층들을 초기화합니다. nn.Module 을 상속받은 모든 클래스는 forward 메소드에 입력 데이터에 대한 연산들을 구현

In [4]:
class NeuralNetworks(nn.Module) :
    def __init__(self) :
        super(NeuralNetworks, self).__init__()
        self.flatten  = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28 , 512) ,
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
             )
        
    def forward(self, x) :
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [5]:
# NeuralNetwork 의 인스턴스(instance)를 생성하고 이를 device 로 이동한 뒤, 구조(structure)를 출력

model = NeuralNetworks().to(device)
print(model)

NeuralNetworks(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


* 모델을 사용하기 위해 입력 데이터를 전달합니다. 이는 일부 백그라운드 연산들 과 함께 모델의 forward 를 실행

* 모델을 사용하기 위해 입력 데이터를 전달합니다. 이는 일부 백그라운드 연산들 과 함께 모델의 forward 를 실행

In [6]:
X = torch.rand(1, 28,28 , device = device)
logits = model(X)

pred_probab = nn.Softmax(dim =1)(logits)
y_pred = pred_probab.argmax(1)

print(f"Predicted class: {y_pred}")

Predicted class: tensor([2])


## 모델 계층 

In [7]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [8]:
# nn.Flatten은 계층을 초기화하여 각 28x28의 2D이미지를 784픽셀 값을 갖는 연속된 배열로 변환 

flatten = nn.Flatten()
flat_impage = flatten(input_image)
print(flat_impage.size())

torch.Size([3, 784])


In [9]:
# 선형 계층은 가중치와 편향을 사용하여 입력에 선형변환을 적용하는 모듈

layer1 = nn.Linear(in_features= 28*28 , out_features= 20)
hidden1 = layer1(flat_impage)
print(hidden1.size())

torch.Size([3, 20])


In [10]:
# nn.Sequential
# 순서를 갖는 모듈의 컨테이너입니다. 
#데이터는 정의된 것과 같은 순서로 모든 모듈들을 통해 전달 됩니다.

seq_models = nn.Sequential(
        flatten,
        layer1,
        nn.ReLU(),
        nn.Linear(20, 10)
)

input_image = torch.rand(3, 28, 28)
logits =seq_models(input_image)



In [11]:
# nn.Softmax
# 신경망의 마지막 선형 계층은 nn.Softmax 모듈에 전달될 ([-infty, infty] 범위의 원시 값(raw value)인) logits 를 반환합니다
# logits는 모델의 각 분류(class)에 대한 예측 확률을 나타내도록 [0, 1] 범위로 비례하여 조정(scale)됩니다

softmax = nn.Softmax(dim = 1)
pred_probab = softmax(logits)

## 모델 매개변수

신경망 내부의 많은 계층들은 매개변수화(parameterize) 됩니다. 즉, 학습 중에 최적화되는 가중치와 편향과 연관지어집니다. nn.Module 을 상속하면 모델 객체 내부의 모든 필드들이 자동으로 추적(track)되며, 모델의 parameters() 및 named_parameters() 메소드로 모든 매개변수에 접근할 수 있게 됩니다.

In [12]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetworks(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0122, -0.0136, -0.0109,  ..., -0.0015,  0.0281,  0.0156],
        [-0.0130,  0.0013,  0.0143,  ..., -0.0229,  0.0075,  0.0208]],
       grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0232,  0.0317], grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0310,  0.0289,  0.0106,  ...,  0.0285,  0.0320, -0.0255],
        [-0.0259,  0.0102,  0.0357,  ..., -0.0124, -0.0144, -0.0022]],
       grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.bias | Si